### Notebook for visualization ground truth dataset

For now, organized with ethograms in dictionary, and videos saved in `/data/caitlin/groud_truth`

In [1]:
import pandas as pd
from ipywidgets import HBox, VBox, Select
from fastplotlib import ImageWidget, Plot
from mesmerize_core.arrays import LazyVideo
from pathlib import Path
import numpy as np
from fastplotlib.graphics.selectors import LinearSelector
from fastplotlib.graphics import ImageGraphic

2023-06-13 14:25:54.557795: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-13 14:25:55.003083: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
ETHOGRAM_COLORS = {
    "lift": "b",
    "handopen": "green",
    "grab": "r",
    "sup": "cyan",
    "atmouth": "magenta",
    "chew": "yellow"
}

In [15]:
df = pd.read_hdf("/data/caitlin/full_hand_labels.hdf")
df

,mat_path,full_hand_labels
0,M234_20170328,"{'M234_20170328_v004': [[0, 0, 0, 0, 0, 0, 0, ..."
1,M234_20170331,"{'M234_20170331_v001': [[0, 0, 0, 0, 0, 0, 0, ..."
2,M234_20170403,"{'M234_20170403_v001': [[0, 0, 0, 0, 0, 0, 0, ..."
3,M234_20170412,"{'M234_20170412_v001': [[0, 0, 0, 0, 0, 0, 0, ..."
4,M234_20170414,"{'M234_20170414_v002': [[0, 0, 0, 0, 0, 0, 0, ..."
...,...,...
84,M326_20201110,"{'M326_20201110_v001': [[0, 0, 0, 0, 0, 0, 0, ..."
85,M328_20201130_2500,"{'M328_20201130_v001': [[0, 0, 0, 0, 0, 0, 0, ..."
86,M336_20210612,"{'M336_20210612_v001': [[0, 0, 0, 0, 0, 0, 0, ..."
87,M336_20210613,"{'M336_20210613_v001': [[0, 0, 0, 0, 0, 0, 0, ..."


In [4]:
ethograms = dict()

In [5]:
for row in df.iterrows():
    keys = row[1]['full_hand_labels'].keys()
    for k in keys:
        ethograms[k] = row[1]['full_hand_labels'][k]

In [6]:
options = [k for k in ethograms.keys()]

In [7]:
len(options)

126

In [8]:
data_path = Path('/data/caitlin/ground_truth/')

In [9]:
def trial_change(obj):
    selected_vid = data_path.joinpath(trial_selector.value).with_suffix('.avi')
    new_data = [LazyVideo(selected_vid)]
    # iw.gridplot[0,0].clear()
    # iw.gridplot[0,0].add_graphic(ImageGraphic(new_data[0][0]))
    iw.set_data(new_data=new_data, reset_vmin_vmax=True)
    plot.clear()
    make_plot(plot)
    plot.auto_scale()

In [10]:
trial_selector = Select(options=options)
trial_selector.observe(trial_change, "value")

In [11]:
iw = ImageWidget(data=[LazyVideo(data_path.joinpath(trial_selector.value).with_suffix('.avi'))])

RFBOutputContext()

/home/clewis7/repos/mesmerize-core/mesmerize_core/arrays/_video.py:87: UserWarning: min not implemented for LazyTiff, returning min of 0th index
  warn("min not implemented for LazyTiff, returning min of 0th index")
/home/clewis7/repos/mesmerize-core/mesmerize_core/arrays/_video.py:92: UserWarning: max not implemented for LazyTiff, returning min of 0th index
  warn("max not implemented for LazyTiff, returning min of 0th index")


In [12]:
def ethogram_event_handler(ev):
    ix = ev.pick_info["selected_index"]
    iw.sliders["t"].value = ix

In [13]:
plot = Plot(size=(500, 100))
def make_plot(plot):
        
    ethogram_array = ethograms[trial_selector.value]
    y_bottom = 0
    for i, b in enumerate(ETHOGRAM_COLORS.keys()):
        xs = np.arange(ethogram_array.shape[1], dtype=np.float32)
        ys = np.zeros(xs.size, dtype=np.float32)

        lg = plot.add_line(
            data=np.column_stack([xs, ys]),
            thickness=10,
            name=b
        )

        lg.colors = 0
        lg.colors[ethogram_array[i] == 1] = ETHOGRAM_COLORS[b]

        y_pos = (i * -10) - 1
        lg.position_y = y_pos

    ethogram_selector = LinearSelector(
                selection=0,
                limits=(0, ethogram_array.shape[1]),
                axis="x",
                parent=lg,
                end_points=(y_bottom, y_pos),
            )

    plot.add_graphic(ethogram_selector)
    ethogram_selector.selection.add_event_handler(ethogram_event_handler)

RFBOutputContext()

In [14]:
make_plot(plot)
VBox([
    HBox([iw.show(), trial_selector]),
    plot.show()
])